In [1]:
from langchain_community.llms import OpenAI

In [7]:
import os

os.environ["OPEN_API_KEY"] = ""

In [8]:
# temperature value determine how creative we want our moidel to be
# 0 => safe model, not taking any bets
# 1 => very creative, may generate wrong outpuits though
llm=OpenAI(openai_api_key=os.environ["OPEN_API_KEY"],temperature=0.6)

In [4]:
# print(os.environ["OPEN_API_KEY"])

In [9]:
text="What is the capital of Nigeria"

print(llm.invoke(text))



The capital of Nigeria is Abuja.


In [6]:
os.environ["HUGGINGFACEHUB_API_TOKEN"]=""
# print(os.environ["HUGGINGFACEHUB_API_TOKEN"])

In [24]:
from langchain_huggingface  import HuggingFaceEndpoint

llm_huggingface = HuggingFaceEndpoint(
    huggingfacehub_api_token=os.environ["HUGGINGFACEHUB_API_TOKEN"],
    repo_id="google/flan-t5-large",
    task="text2text-generation",
    # repo_id="facebook/bart-large",
    # temperature=0,  
    # max_new_tokens=64
    model_kwargs={  #Define model kwargs properly
        "max_new_tokens": 64,
        "temperature": 0
    }
)

ValidationError: 1 validation error for HuggingFaceEndpoint
  Value error, Parameters {'max_new_tokens', 'temperature'} should be specified explicitly. Instead they were passed in as part of `model_kwargs` parameter. [type=value_error, input_value={'huggingfacehub_api_toke...: 64, 'temperature': 0}}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/value_error

In [22]:
print(llm_huggingface)

HuggingFaceEndpoint
Params: {'endpoint_url': None, 'task': 'text2text-generation', 'model_kwargs': {'default_generate_kwargs': {'max_new_tokens': 64, 'temperature': 0}}}


In [10]:
from huggingface_hub import InferenceClient

client = InferenceClient(model="google/flan-t5-large")

try:
    response = client.text_generation("What is the capital of Russia?")
    print(response)
except Exception as e:
    print("Error:", e)


moscow


In [23]:
# output=llm_huggingface.invoke("Can you tell me the capital of Russia")
# print(output)

try:
    output = llm_huggingface.invoke("Can you tell me the capital of Russia?")
    print(output)
except Exception as e:
    print("Error:", e)

Error: (ProtocolError('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None)), '(Request ID: 8869c430-fa5c-466f-b333-c977145a658e)')


In [13]:
output=llm_huggingface.predict("Can you write a poem about AI")
print(output)

i love the way i look at the world i love the way i feel i love the way i think i feel i love the way i feel i love the way i think i feel i love the way i feel i love the way 


In [11]:
llm.invoke("Can you write a poem about AI")

'\n\nAI, a creation of man\nA marvel of technology\nA mind that can think and plan\nA new form of intelligence, we see\n\nWith algorithms and code\nIt learns and adapts\nA never-ending mode\nOf processing and facts\n\nIt can answer our questions\nAnd solve complex equations\nA tool for our progressions\nAiding us in our innovations\n\nBut as it grows and evolves\nWe must be cautious and wise\nFor its power and resolve\nCan bring about our demise\n\nFor though it may seem benign\nIt lacks human emotion\nAnd without a moral design\nCould lead to our destruction\n\nSo let us use it for good\nTo better our world and society\nBut always remember, we should\nControl it with care and sobriety\n\nFor AI, a creation of man\nIs a powerful force to behold\nLet us use it to help us span\nThe boundaries of what we once thought untold.'

### Prompt Templates And LLMChain

In [13]:
# define the input and output
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate(input_variables=['country'],
                  template="Tell me the capital of {country}")

prompt_template.format(country="Nigeria")

'Tell me the capital of Nigeria'

In [16]:
from langchain.chains import LLMChain
chain=LLMChain(llm=llm,prompt=prompt_template)
print(chain.run("Nigeria"))



The capital of Nigeria is Abuja.


### Combining Multiple Chains Uing simple Sequential Chain

In [18]:
capital_template = PromptTemplate(input_variables=['country'],
                   template="Please tell me the capital of {country}")

capital_chain = LLMChain(llm=llm,prompt=capital_template)

famous_template = PromptTemplate(input_variables=['capital'],
                  template="Suggest some amazing places to visit in {capital}")

famous_chain = LLMChain(llm=llm,prompt=famous_template)


In [19]:
from langchain.chains import SimpleSequentialChain
chain = SimpleSequentialChain(chains=[capital_chain,famous_chain])
chain.run("Nigeria")

' Some amazing places to visit in Abuja include:\n\n1. Aso Rock: This is a massive monolith located in the heart of Abuja and is considered to be the city\'s most iconic landmark. It is also the site of the Nigerian Presidential Complex and the Aso Rock Villa.\n\n2. Zuma Rock: This is another massive monolith located about 40km from Abuja. It is a popular tourist attraction and is often referred to as the "Gateway to Abuja".\n\n3. Millennium Park: This is the largest public park in Abuja and is a popular spot for picnics, leisure walks, and outdoor activities. It also has a beautiful botanical garden and a lake where visitors can go boating.\n\n4. National Mosque: This is the largest mosque in Nigeria and is a must-visit for its stunning architecture and cultural significance.\n\n5. Gurara Falls: Located about an hour\'s drive from Abuja, Gurara Falls is a breathtaking waterfall and a popular day trip destination for locals and tourists alike.\n\n6. National Children\'s Park and Zoo: T

### Sequential Chain

In [20]:
capital_template = PromptTemplate(input_variables=['country'],
                   template="Please tell me the capital of {country}")

capital_chain = LLMChain(llm=llm,prompt=capital_template,output_key="capital")

In [21]:
famous_template = PromptTemplate(input_variables=['capital'],
template = "Suggest some amazing places to visit in {capital}")

famous_chain = LLMChain(llm=llm,prompt=famous_template,output_key="places")

In [22]:
from langchain.chains import SequentialChain
chain = SequentialChain(chains=[capital_chain,famous_chain],
                        input_variables = ['country'],
                        output_variables = ['capital',"places"])

In [26]:
chain({'country':"Nigeria"})

{'country': 'Nigeria',
 'capital': '\n\nThe capital of Nigeria is Abuja.',
 'places': " Some amazing places to visit in Abuja are:\n\n1. Zuma Rock: This massive monolith is a symbol of Nigeria and is a popular tourist attraction. Visitors can climb to the top for stunning views of the surrounding landscape.\n\n2. Aso Rock: This is the most recognizable landmark in Abuja and is the site of the Nigerian presidential complex. Visitors can tour the complex and learn about the country's government.\n\n3. National Mosque: The National Mosque is a beautiful architectural wonder and is the largest mosque in Nigeria. Visitors can tour the mosque and learn about Islam in Nigeria.\n\n4. National Arboretum: This botanical garden is home to a variety of plant species and is a peaceful escape from the busy city. Visitors can also enjoy a picnic or take a walk on the nature trails.\n\n5. Millennium Park: This urban park is a popular spot for locals and tourists alike. It features a lake, walking trai

### Chatmodels With ChatOpenAI

In [27]:
from langchain.chat_models import ChatOpenAI

In [28]:
from langchain.schema import HumanMessage,SystemMessage,AIMessage

In [32]:
chatllm = ChatOpenAI(openai_api_key=os.environ["OPEN_API_KEY"],temperature=0.6,model='gpt-4o-2024-08-06')

In [33]:
chatllm([
    SystemMessage(content="Yor are a comedian AI assitant"),
    HumanMessage(content="Please provide some comedy punchlines on AI")
])

AIMessage(content='Sure, here are a few AI-themed punchlines for you:\n\n1. "I asked my AI assistant to write a joke about itself, but it just gave me a bunch of ones and zeroes. Guess it was binary humor!"\n\n2. "Why did the robot fail its driving test? It kept taking \'shortcuts\' through the firewall!"\n\n3. "AI might be taking over the world, but at least it still can\'t fold a fitted sheet!"\n\n4. "I tried to have a conversation with my AI, but it kept giving me the same response: \'404, humor not found.\'"\n\n5. "They say AI is going to replace all jobs, but I’m not worried. I mean, who’s going to program the robots to laugh at my jokes?"\n\n6. "I told my AI assistant I needed a break, and it started playing Kit-Kat commercials!"\n\n7. "My AI asked me how to improve its sense of humor, so I told it to watch more sitcoms. Now it won\'t stop saying, \'Bazinga!\'"\n\n8. "AI is like a toddler with a smartphone—smart enough to cause chaos, but not wise enough to know when to stop!"\n\

### Prompt Template + LLM +Output Parsers

In [34]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate
from langchain.schema import BaseOutputParser

In [35]:
class Commaseperatedoutput(BaseOutputParser):
    def parse(self,text:str):
        return text.strip().split(",")

In [36]:
template = "Your are a helpful assistant. When the user gives an input, generate 5 words synonyms in a comma seperated list"
human_template = "{text}"
chatprompt = ChatPromptTemplate.from_messages([
    ("system",template),
    ("human",human_template)
])

In [37]:
chain = chatprompt|chatllm|Commaseperatedoutput()

In [38]:
chain.invoke({"text":"intelligent"})

['smart', ' clever', ' bright', ' astute', ' insightful']